In [2]:
import utils
import numpy as np
from collections import deque

In [5]:


def region_growing(im: np.ndarray, seed_points: list, T: int) -> np.ndarray:
    """
    Region growing segmentation using 8-connected neighborhood and intensity homogeneity.

    Args:
        im: Grayscale image of shape (H, W), dtype=np.uint8, values in [0, 255].
        seed_points: list of [row, col] seed coordinates (0-based).
        T: non-negative integer threshold for homogeneity criterion.

    Returns:
        np.ndarray of shape (H, W), dtype=bool. True where segmented (foreground).
    """
    assert im.dtype == np.uint8, "im must be uint8"
    assert T >= 0, "T must be non-negative"

    H, W = im.shape
    segmented = np.zeros((H, W), dtype=bool)
    # Per-seed visited map to avoid looping; we also avoid re-adding pixels
    visited = np.zeros((H, W), dtype=bool)

    # 8-connected (Moore) neighborhood offsets
    neighbours = [(-1, -1), (-1, 0), (-1, 1),
            ( 0, -1),          ( 0, 1),
            ( 1, -1), ( 1, 0), ( 1, 1)]

  
    for seed in seed_points:
        seed_x, seed_y = seed
     
        # skip invalid seeds
        if not (0 <= seed_x < H and 0 <= seed_y < W):
            continue

        # already covered by a previous seed's region
        if segmented[seed_x, seed_y]:
            continue

        seed_intensity = int(im[seed_x, seed_y])
        queue = deque()
        queue.append((seed_x, seed_y))
        visited[seed_x, seed_y] = True
        segmented[seed_x, seed_y] = True
        while queue:
            # Get current pixel
            seed_x, seed_y = queue.popleft()
            # Check all 8-connected neighbors
            for dx, dy in neighbours:
                # Neighbor coordinates
                nx, ny = seed_x + dx, seed_y + dy

                # Check bounds
                if not (0 <= nx < H and 0 <= ny < W):
                    continue

                # Check if already visited
                if visited[nx, ny]:
                    continue

                # Check homogeneity criterion
                neighbor_intensity = int(im[nx, ny])
                if abs(neighbor_intensity - seed_intensity) <= T:
                    segmented[nx, ny] = True
                    visited[nx, ny] = True
                    queue.append((nx, ny))

    return segmented


In [6]:
if __name__ == "__main__":
    # DO NOT CHANGE
    im = utils.read_image("defective-weld.png")

    seed_points = [ # (row, column)
        [254, 138], # Seed point 1
        [253, 296], # Seed point 2
        [233, 436], # Seed point 3
        [232, 417], # Seed point 4
    ]
    intensity_threshold = 50
    segmented_image = region_growing(im, seed_points, intensity_threshold)

    assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
        im.shape, segmented_image.shape)
    assert segmented_image.dtype == bool, "Expected thresholded image dtype to be bool. Was: {}".format(
            segmented_image.dtype)

    segmented_image = utils.to_uint8(segmented_image)
    utils.save_im("defective-weld-segmented.png", segmented_image)


Reading image: images\defective-weld.png
Saving image to: image_processed\defective-weld-segmented.png


c:\Users\adeshpande\AppData\Local\Programs\Python\Python312\Lib\site-packages\skimage\_shared\utils.py:328: UserWarning: C:\Users\adeshpande\Documents\Uni\TDT4195-Grunnleggende-Visuell-Databehandling\Assignment 6\image_processed\defective-weld-segmented.png is a low contrast image
  return func(*args, **kwargs)
